<a href="https://colab.research.google.com/github/zh79325/jxh-ai-server/blob/main/hub-sku110k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a align="center" href="https://ultralytics.com/hub" target="_blank">
<img width="1024", src="https://github.com/ultralytics/assets/raw/main/im/ultralytics-hub.png"></a>

<div align="center">

[中文](https://docs.ultralytics.com/zh/hub/) | [한국어](https://docs.ultralytics.com/ko/hub/) | [日本語](https://docs.ultralytics.com/ja/hub/) | [Русский](https://docs.ultralytics.com/ru/hub/) | [Deutsch](https://docs.ultralytics.com/de/hub/) | [Français](https://docs.ultralytics.com/fr/hub/) | [Español](https://docs.ultralytics.com/es/hub/) | [Português](https://docs.ultralytics.com/pt/hub/) | [Türkçe](https://docs.ultralytics.com/tr/hub/) | [Tiếng Việt](https://docs.ultralytics.com/vi/hub/) | [العربية](https://docs.ultralytics.com/ar/hub/)

  <a href="https://github.com/ultralytics/hub/actions/workflows/ci.yaml"><img src="https://github.com/ultralytics/hub/actions/workflows/ci.yaml/badge.svg" alt="CI CPU"></a>
  <a href="https://colab.research.google.com/github/ultralytics/hub/blob/main/hub.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>

Welcome to the [Ultralytics](https://ultralytics.com/) HUB notebook!

This notebook allows you to train Ultralytics [YOLO](https://github.com/ultralytics/ultralytics) 🚀 models using [HUB](https://hub.ultralytics.com/). Please browse the HUB <a href="https://docs.ultralytics.com/hub/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/hub/issues/new/choose">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!
</div>

# 环境初始化

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
%pip install ultralytics  # install
from ultralytics import YOLO, checks, hub
from ultralytics.utils.files import WorkingDirectory
from ultralytics.utils.downloads import download
import shutil
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
checks()  # checks

current = os.path.dirname(os.path.abspath('__file__'))
parent = Path(current+'/codes')
if  parent.exists():
  shutil.rmtree(parent)
os.makedirs(parent)
download('https://github.com/zh79325/jxh-ai-server/archive/refs/heads/main.zip',dir=parent)



Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (AMD EPYC 7B13)
Setup complete ✅ (24 CPUs, 47.1 GB RAM, 15.5/225.3 GB disk)


293MB [00:16, 19.0MB/s]
Unzipping /content/codes/main.zip to /content/codes/jxh-ai-server-main...: 100%|██████████| 63/63 [00:01<00:00, 32.62file/s]


# 下载人工打标数据集


In [ ]:
import os.path
import shutil
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywh
import yaml

current = os.path.dirname(os.path.abspath('__file__'))
dir = Path(current+'/sku110k')  # dataset root dir
parent = Path(dir.parent)  # download dir
checks()  # checks
urls = ['http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz']
download(urls, dir=parent)
# Rename directories
if dir.exists():
    shutil.rmtree(dir)
(parent / 'SKU110K_fixed').rename(dir)  # rename dir
(dir / 'labels').mkdir(parents=True, exist_ok=True)  # create labels dir

def formatN(n):
    if n < 0:
        return 0
    elif n > 1:
        return 1
    else:
        return n
# Convert labels
names = 'image', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height'  # column names
for d in 'annotations_train.csv', 'annotations_val.csv', 'annotations_test.csv':
    x = pd.read_csv(dir / 'annotations' / d, names=names).values  # annotations
    images, unique_images = x[:, 0], np.unique(x[:, 0])
    with open((dir / d).with_suffix('.txt').__str__().replace('annotations_', ''), 'w') as f:
        f.writelines(f'./images/{s}\n' for s in unique_images)
    for im in tqdm(unique_images, desc=f'Converting {dir / d}'):
        cls = 0  # single-class dataset
        with open((dir / 'labels' / im).with_suffix('.txt'), 'a') as f:
            for r in x[images == im]:
                w, h = r[6], r[7]  # image width, height
                xywh = xyxy2xywh(np.array([[r[1] / w, r[2] / h, r[3] / w, r[4] / h]]))[0]  # instance
                f.write(f"{cls} {formatN(xywh[0]-xywh[2]/2):.5f} {formatN(xywh[1]-xywh[3]/2):.5f} {formatN(xywh[0]+xywh[2]-xywh[2]/2):.5f} {formatN(xywh[1]-xywh[3]/2):.5f} {formatN(xywh[0]+xywh[2]-xywh[2]/2):.5f} {formatN(xywh[1]+xywh[3]-xywh[3]/2):.5f} {formatN(xywh[0]-xywh[2]/2):.5f} {formatN(xywh[1]+xywh[3]-xywh[3]/2):.5f}\n")  # write label


Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (AMD EPYC 7B13)
Setup complete ✅ (24 CPUs, 47.1 GB RAM, 16.1/225.3 GB disk)


 17%|█▋        | 1.92G/11.4G [00:28<02:55, 57.8MB/s]

# 模型训练

In [ ]:
import os
import shutil
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import fontManager
import yaml
from ultralytics import YOLO
root = current+'/codes/jxh-ai-server-main/jxh-ai-models'
mpl_data_path = mpl.get_data_path()
mpl_fonts_folder = mpl_data_path + '/fonts/ttf'
print("get_data_path=>", mpl.get_data_path())
if not os.path.exists(mpl_fonts_folder + '/SimHei.ttf'):
    shutil.copyfile(root + '/SimHei.ttf', mpl_fonts_folder + '/SimHei.ttf')
fontManager.addfont(mpl_fonts_folder + '/SimHei.ttf')
plt.rcParams["font.family"] = "SimHei"
plt.rcParams['font.sans-serif'] = ['SimHei']  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  #用来正常显示负号

model_yaml = os.path.abspath(f'{current}/sku110k.yaml')
labelList = ['object']
data = dict(
    path=current+'/sku110k',
    train='train.txt',
    val='val.txt',
    test='test.txt',
    nc=len(labelList),
    names=labelList
)
with open(model_yaml, 'w') as outfile:
  yaml.dump(data, outfile, default_flow_style=False)
init_model=root+'/models/input/yolo11l-obb.pt'
print('load init_model from =>',init_model)
print('load model_yaml from =>',model_yaml)
model = YOLO(init_model)  # build a new model from YAML
results = model.train(data=model_yaml,batch=4, epochs=100, imgsz=640)

get_data_path=> /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data
load init_model from => /content/codes/jxh-ai-server-main/jxh-ai-models/models/input/yolo11l-obb.pt
load model_yaml from => /content/sku110k.yaml
Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.5.1+cpu CPU (AMD EPYC 7B13)
engine/trainer: task=obb, mode=train, model=/content/codes/jxh-ai-server-main/jxh-ai-models/models/input/yolo11l-obb.pt, data=/content/sku110k.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, s

train: Scanning /content/sku110k/labels... 8216 images, 0 backgrounds, 3 corrupt: 100%|██████████| 8219/8219 [01:49<00:00, 74.95it/s]

train: WARNING ⚠️ /content/sku110k/images/train_1797.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /content/sku110k/images/train_4222.jpg: ignoring corrupt image/label: image file is truncated (23 bytes not processed)
train: WARNING ⚠️ /content/sku110k/images/train_882.jpg: ignoring corrupt image/label: image file is truncated (3 bytes not processed)
train: WARNING ⚠️ /content/sku110k/images/train_924.jpg: ignoring corrupt image/label: image file is truncated (10 bytes not processed)


train: New cache created: /content/sku110k/labels.cache


val: Scanning /content/sku110k/labels... 588 images, 0 backgrounds, 0 corrupt: 100%|██████████| 588/588 [00:06<00:00, 96.29it/s]


val: New cache created: /content/sku110k/labels.cache
Plotting labels to runs/obb/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.0005), 182 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/obb/train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      2.578      3.523      2.771       1255        640:   0%|          | 5/2054 [00:18<2:04:24,  3.64s/it]

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
       print(f"GPU index: {torch.cuda.current_device()}")

if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")
        for i in range(num_gpus):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


Using device: cuda
GPU index: 0
Number of GPUs available: 1
GPU 0: Tesla T4
